# Illustration of the Quantum Linear Systems Algorithm

#### This code is an illustration of how the Quantum Linear Systems of Equations Algorithm [Harrow, Hassidim, Lloyd 2009 PRL] stepwise manipulates the quantum state of a 1+tau+n - qubit system. 
Note: The implementation is NOT a classical simulation of the elementary quantum operations themselves, but is supposed to have a didactical purpose

In [1]:
import sys
import numpy as np
import math as math
from tabulate import tabulate # for nice printing

#### The class qustate defines the quantum state, how to manipulate it and how to print it

In [3]:
class QuState:
    
    def __init__(self, tau, n): # create an initial quantum state in which all qubits are in 0
        self.qu_state = [[1, ''.zfill( 1+tau+n )]]
        self.n = n # number of qubits to represent b and x
        self.tau = tau # number of qubits for the eigenvalue register
        
    # Query if a basis state 'exists' (i.e., the event has finite probability)   
    def contains(self, bstate): 
        if bstate in self.get_bstate_list():
            return True
        else:
            return False
    
    def create(self, amplitude, bstate): # Create a basis state (i.e., a previously zero amplitude gets a nonzero value)
        if self.contains(bstate):
            self.overwrite(amplitude, bstate)
        else:
            self.qu_state.append([amplitude, bstate])
    
    def delete(self, bstate): # Set a previously nonzero amplitude to zero
        self.qu_state.remove( self.qu_state[self.get_bstate_list().index(bstate)] )
              
    def overwrite(self, amplitude, bstate): # Change the value of a nonzero amplitude 
        self.qu_state[self.get_bstate_list().index(bstate)][0] = amplitude
        
    def addto(self, summand, bstate): # Add something to the value of a nonzero amplitude 
        if self.contains(bstate):
            self.qu_state[self.get_bstate_list().index(bstate)][0] += summand
        else:
            self.create(summand, bstate) 
            
    def multiplyto(self, factor, bstate): # Multiply something to the value of a nonzero amplitude 
        if self.contains(bstate):
            self.qu_state[self.get_bstate_list().index(bstate)][0] *= factor
        else:
            self.create(0, bstate)         
            
    def get_amplitude(self, bstate): #Return the amplitude associated with a given basis state
        if self.contains(bstate):
            return self.qu_state[self.get_bstate_list().index(bstate)][0]
        else: 
            print ("Error in 'get_amplitude': no such state found!")
            return -99

    def sort_qu_state(self): # Sort the list according to basis states for printing
        self.qu_state = sorted(self.qu_state, key=lambda qu_state: qu_state[1])
            
    def print_me(self, step): # Print current quantum state: First column is the index of the amplitude in the 2^{(1+tau+n)} dimensional amplitude vector.
        # The indices in the table are all entries with a nonzero amplitude. 
        self.sort_qu_state() 
        printable_qu_state = [ [int(qs[1],2), qs[0], qs[1][0] + ' ' + qs[1][1:self.tau+1] + ' ' + qs[1][self.tau+1: self.n+self.tau +1]] for qs in self.qu_state] 
        if (step != 0):
            print 
            print ("After STEP ", step,":")
            print
        print(tabulate(printable_qu_state, headers=["Index","Amplitude","Basis State"])  )    
              
    def get_amplitude_list(self):
        return [qs[0] for qs in self.qu_state]
    
    def get_bstate_list(self):
        return [qs[1] for qs in self.qu_state]

    def renormalise(self): # Renormalise the quantum state so that \sum_i |amplitude_i|^2 = 1 (which should always be the case) 
        av = self.get_amplitude_list()
        nf = float(np.sqrt(np.dot(av,av)))
        for qs in self.qu_state:
            qs[0] = qs[0]/nf 
        return nf

## More functions needed

#### Create the right type of positive definite square matrices as inputs 

In [4]:
def preprocess(mat, vec):

    if (mat.shape[0] != mat.shape[1]) or (mat.shape[0] != len(vec)):
        print("PLEASE MAKE SURE A IS SQUARE AND USE MATCHING INPUT DIMENSIONS FOR A AND b!")
        sys.exit(1)
        
    if (not isPositiveDefinite(mat)): 
        print("PLEASE ENTER A POSITIVE DEFINITE MATRIX")
        sys.exit(1)
        
    ### If b has not dimension 2**n for an integer n, define larger LSE --> Read the solution from the first N entries of x
    nn = math.log(len(vec),2)
    if (nn%1 != 0):
        vec = np.array([vec[i] if i<len(vec) else 0  for i in range(int(nn+1)**2)] )
        mat = np.array([[ mat[i,j] if (i<mat.shape[0] and j<mat.shape[0]) else 0  for i in range(int(nn+1)**2)] for j in range(int(nn+1)**2)])
       

    ### If b is not Hermitian, define larger LSE --> Read the solution from the first N entries of x
    if (not isHermitian(mat)):
        zero_block = np.array([[0 for i in range(len(mat))] for j in range(len(mat))])
        mat = np.array(np.bmat([[mat, zero_block] , [zero_block, mat]]))
        vec = np.hstack([vec, zero_block[0]])
        
    eigenvals, eigenvecs = np.linalg.eig(mat)
  
    ### Rescale A,b so that A's eigenvalues are in [0,1] --> Should not affect the solution
    if (eigenvals.max() > 1):
        mat = np.array(mat)/eigenvals.max()
        vec = np.array(vec)/eigenvals.max()
        eigenvals, eigenvecs = np.linalg.eig(mat)

    ### Normalise b and remember normalisation factor
    if (not isNormalised(vec)):
        nf = float(np.sqrt(np.dot(vec,vec)))
        vec = vec/nf
    else:
        nf = 1

    ### Check once more if new matrix is positive definite
    if (not isPositiveDefinite(mat)): 
        print("PREPROCESSING PRODUCED A NON-POSITIVE DEFINITE MATRIX. PLEASE TRY A DIFFERENT INPUT.")
        sys.exit(1)

    return mat, vec, nf

#### And more functions...

In [5]:
def isHermitian(matrix):
    if (matrix ==  np.conjugate(np.transpose(matrix))).all():
        return True
    else:
        return False
        
def isNormalised(vector):
    if (np.dot(vector,np.conjugate(vector)) == 1.):
        return True
    else:
        return False   
        
def isPositiveDefinite(mat):
    eigenvals, eigenvecs = np.linalg.eig(mat)
    if any(item <= 0 for item in eigenvals):
        return False
    else:
        return True

def index_to_bstate(index, number_of_bits): # Convert integer to binary
    return format(index, '0'+str(number_of_bits)+'b')
    
def bstate_to_index(state): # Convert binary to integer
    return int(state, 2)

def bin_to_evl(bin_evl): # Convert a binary to a real number [0,1] 
    return sum([int(bin_evl[i])*(1/2.**(i+1)) for i in range(len(bin_evl))])
    
    
def evl_to_bin(evl, length): # Convert a real number in [0,1] to a binary (this is how the eigenvalues are encoded)
    temp = evl
    bits = ''
    for i in range(length):
        if (temp - (1/2.**(i+1)) >= 0):
            temp = temp - (1/2.**(i+1))
            bits += '1'
        else: 
            bits += '0'
    return bits

## Now the algorithm starts

### Goal: Solve Ax = b where A is a NxN matrix and b a vector of dimension N

 The quantum algorithm requires A to be Hermitian, have positive eigenvalues in ]0,1], b to be unit-length, 
 and N = 2^n for an integer n (because b is represented by the 2^n amplitudes of n qubits)
 We can redefine Ax = b so that everything fits and postprocess the solution accordingly
 !!!!(NB: Only implemented for real eigenvalues or positive definite matrices for now.) !!!!
 
 --> Check the HHl paper for details.

In [7]:

A_init = np.array([[2., 1.],[1.,2.]])  
b_init = np.array([0.2 , 0.7]) #
tau = 10 # Choose the number of qubits (precision) to represent eigenvalues in the quantum algo

# Produce a Hermitian matrix A of dimension 2^n x 2^n (n integer) with eigenvalues in ]0,1]
# and a unit-length vector b of dimension 2^n out of the original inputs A and b 
A,b, nf1 = preprocess(A_init, b_init)

# Some stuff needed for the classical simulation which the quantum system does generically :
eigenvals, eigenvecs = np.linalg.eig(A)  # The eigendecomposition of A
eigenvecs = np.transpose(eigenvecs)  # Needed because numpy returns the eigenvectors as columns of eigenvecs
bin_eigenvals = [evl_to_bin(evl, tau) for evl in eigenvals] # A's eigenvalues in binary representation
n = int(math.log(len(b),2)) # Calculate number of qubits needed to represent b 
C = eigenvals.min()/2. # Define a number C that is smaller than the smallest eigenvalue (which has to be estimated somehow) 


# Some printing
np.set_printoptions(precision=3)
print("\n INPUTS (after preprocessing): ")
print( tabulate([['tau' , tau],  ['A',  np.ndarray.tolist(A)], ['b', b], ['n' , n]]))
print( "\n SPECTRAL PROPERTIES OF A: ")
print( tabulate([['Eigenvalues of A',eigenvals] ,
['Eigenvalues in binary representation ' ,bin_eigenvals],['Error of binary representation' , [abs(bin_to_evl(bin_eigenvals[i])-eigenvals[i]) for i in range(len(bin_eigenvals))]], 
['Eigenvectors ', np.ndarray.tolist(eigenvecs) ]]))


 INPUTS (after preprocessing): 
---  ------------------------------------------------------------------------------------
tau  10
A    [[0.6666666666666666, 0.3333333333333333], [0.3333333333333333, 0.6666666666666666]]
b    [ 0.275  0.962]
n    1
---  ------------------------------------------------------------------------------------

 SPECTRAL PROPERTIES OF A: 
------------------------------------  -------------------------------------------------------------------------------------
Eigenvalues of A                      [ 1.     0.333]
Eigenvalues in binary representation  ['1111111111', '0101010101']
Error of binary representation        [0.0009765625, 0.00032552083333331483]
Eigenvectors                          [[0.7071067811865475, 0.7071067811865475], [-0.7071067811865475, 0.7071067811865475]]
------------------------------------  -------------------------------------------------------------------------------------


### STEP 1: STATE PREPARATION  
[HHL: page 2, column 2, center]

In [8]:

quantum_state = QuState(tau, n) # Create a quantum state of 1+tau+n qubit initially all in state zero 

for i in range(len(b)): # create basis states  '0 0...0 0...0' to '0 0...0 1...1' with amplitudes b_1 ... b_N
    quantum_state.create(b[i], ''.zfill( 1+tau ) + index_to_bstate(i, n)) 

quantum_state.print_me(1) # Print quantum state after Step 1


After STEP  1 :
  Index    Amplitude  Basis State
-------  -----------  --------------
      0     0.274721  0 0000000000 0
      1     0.961524  0 0000000000 1


### STEP 2: QUANTUM SIMULATION AND QUANTUM PHASE ESTIMATION
[HHL: page 2, column 2, bottom]
This step reproduces the effect of a rather complex subroutine of applying an operator e^(iAt) to the state and then executing a quantum phase estimation algorithm, which is very difficult to properly display in a classical simulation. Therefore, we just update the amplitudes to how 
they look like afterwards (\sum_j \sum_i (u_j*b) (u_j)_i|0>|\lambda_j>|i> ).

Effectively this splits up the sums and writes every (u_j*b)(u_j)_i term into a different position of the amplitude vector 
(the position is defined by the binary representation of the j'th eigenvalue)

In [9]:

# First, delete the old amplitudes (ugly implementation, I know)
for i in range(len(b)): 
    quantum_state.delete(''.zfill( 1+tau ) + index_to_bstate(i, n))
    
# Write the new amplitudes after the quantum routine into the state
for j in range(len(eigenvals)):
    for i in range(len(b)): 
        quantum_state.addto( np.dot(eigenvecs[j],b)* eigenvecs[j][i], '0' + evl_to_bin(eigenvals[j], tau) + index_to_bstate(i, n) )

quantum_state.print_me(2)


After STEP  2 :
  Index    Amplitude  Basis State
-------  -----------  --------------
    682    -0.343401  0 0101010101 0
    683     0.343401  0 0101010101 1
   2046     0.618123  0 1111111111 0
   2047     0.618123  0 1111111111 1


### STEP 3: Conditional Rotation of Ancilla
[HHL: page 3, column 1,  third equation]

 This subroutine 'splits' every term of the superposition into the 'existing' basis state (where the ancilla is 0) and otherwise identical basis state where the ancilla is 1. The proportions of this 'splitting' are conditioned on the eigenvalue-register bits (so that the amplitude of the basis state with ancilla in 1 is chosen to be the inverse eigenvalue times a normalisation factor C necessary to make sure we are in [0,1]).

In [10]:
qu_state_copy = [qs for qs in quantum_state.qu_state] # copy to iterate
for qs in qu_state_copy: #For all terms in the superposition
    bstate = qs[1] # Get the basis state
    ampl = qs[0] # Get the amplitude
    eigval = bin_to_evl(bstate[1:tau+1]) # Get the eigenvalue encoded in the eigenvalue-register
    quantum_state.multiplyto(np.sqrt(1 - (C**2/ eigval**2) ), bstate  ) # Change the amplitude in the original basis state
    quantum_state.create(ampl*(C / eigval) , '1' +  bstate[1:len(bstate)]  ) # Create a state that has the first (ancilla) qubit flipped
    
quantum_state.print_me(3)

After STEP  3 :
  Index    Amplitude  Basis State
-------  -----------  --------------
    682    -0.297297  0 0101010101 0
    683     0.297297  0 0101010101 1
   2046     0.60946   0 1111111111 0
   2047     0.60946   0 1111111111 1
   2730    -0.171869  1 0101010101 0
   2731     0.171869  1 0101010101 1
   4094     0.103121  1 1111111111 0
   4095     0.103121  1 1111111111 1


### STEP 4: Uncomputing the eigenvalue-register
[HHL: page 3, column 1,  First sentence]

 Reverses the Quantum Phase Estimation so that the eigenvalue-register is back to |0...0>.  Here this means to add all amplitudes of basis states where the eigenvalue register is not '0...0' to the corresponding basis state where the eigenvalue register is in '0...0'.

In [11]:
qu_state_copy = [qs for qs in quantum_state.qu_state] # copy to iterate
for qs in qu_state_copy:
    bstate = qs[1]
    ampl = qs[0]
    quantum_state.addto(ampl , bstate[0] + ''.zfill(tau) + bstate[ tau+1 : len(bstate)])
        
qu_state_copy = [qs for qs in quantum_state.qu_state] # copy to iterate
for qs in qu_state_copy:
    if qs[1][1:tau+1] != ''.zfill(tau):
        quantum_state.delete(qs[1]) 
quantum_state.renormalise()

quantum_state.print_me(4)

After STEP  4 :
  Index    Amplitude  Basis State
-------  -----------  --------------
      0    0.312163   0 0000000000 0
      1    0.906757   0 0000000000 1
   2048   -0.0687474  1 0000000000 0
   2049    0.27499    1 0000000000 1


### STEP 5: Conditional measurement of ancilla - found in 1 with certainty
[HHL: page 3, column 1,  fourth equation]

 Measures the first qubit and repeats the entire quantum routine until we found it in state '1' (which of course updates our knowledge equivalent to a classical distribution). Remembers the normalisation factor which can in reality be extracted from the chance of success to measure 1.

In [12]:
qu_state_copy = [qs for qs in quantum_state.qu_state] # copy to iterate
for qs in qu_state_copy:
    bstate = qs[1]
    if bstate[0] == '0': # delete all basis states in which the ancilla appears in '0'
        quantum_state.delete(bstate)        
normalisationFactor = quantum_state.renormalise()
quantum_state.print_me(5)

After STEP  5 :
  Index    Amplitude  Basis State
-------  -----------  --------------
   2048    -0.242536  1 0000000000 0
   2049     0.970143  1 0000000000 1


### STEP 6: POSTPROCESSING AND RESULT

Get the final vector of amplitudes which is equal to x up to a normalisation factor.
Note that just as with classical stochastic systems we would have to rerun the algorithm and sample to get the full amplitude vector, which is why the algorithm is intended not to output a full description of x, but  desired properties extracted in further steps (i.e. any quantum observable). 

In [13]:
final_state = np.array(quantum_state.get_amplitude_list())
# Here we extract the final state and postprocess it in order to compare it to 
# the classical result
nf2 = np.sqrt( 1/ (sum([ np.dot(eigenvecs[j],b)**2 / eigenvals[j]**2   for j in range(len(eigenvals))] ) ) )
result = final_state  / nf2

## PRINT THE RESULTS

In [15]:
print
print( "RESULTS: --------------------------------------\n")
print( "Result of the quantum algorithm x = ", result , "\n") 
print( 'Classical Solution (linalg.solve): x = ' , np.linalg.solve(A, b))
print( 'Classical Solution (manual SVD): x = ' , [ sum([ 1/eigenvals[k] * \
                                                        np.dot(eigenvecs[k],b) * \
                                                        eigenvecs[k][i] \
                                                        for k in range(len(eigenvals)) ])  \
                                                  for i in range(len(b))]  )
print( 'Classical Solution (manual SVD using the approximate Evals): x = ' , \
      [ sum([ 1/bin_to_evl(bin_eigenvals[k]) * np.dot(eigenvecs[k],b) * eigenvecs[k][i] \
             for k in range(len(eigenvals)) ]) \
       for i in range(len(b))] )


RESULTS: --------------------------------------

Result of the quantum algorithm x =  [-0.412  1.648] 

Classical Solution (linalg.solve): x =  [-0.412  1.648]
Classical Solution (manual SVD): x =  [-0.41208169184606713, 1.6483267673842681]
Classical Solution (manual SVD using the approximate Evals): x =  [-0.41248450874914233, 1.6499380349965693]


NOOTE: To get the solution of the initial LSE A_init x = b_init, multiply the resulting vector by the normalisation factor nf1
and consider only the first N entries.
